<a href="https://colab.research.google.com/github/Gnkhakimova/CS5590-Cyber/blob/master/Copy_of_L4_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [49]:
!pip install syft
!pip install numpy

In [0]:
import torch as th

In [76]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [78]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [0]:
import syft as sy

In [80]:
hook = sy.TorchHook(th)  # this line of code create an instance of th with its backend APIs modified with Syft functions

W0625 23:16:30.646510 140450477549440 hook.py:97] Torch was already hooked... skipping hooking process


In [81]:
th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [83]:
bob._objects

{77944742512: tensor([1, 1, 1, 1, 1]), 90764161042: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob) # send this data to bob

In [86]:
x_ptr

(Wrapper)>[PointerTensor | me:86663614844 -> bob:67898169666]

In [87]:
bob._objects

{67898169666: tensor([1, 2, 3, 4, 5]),
 77944742512: tensor([1, 1, 1, 1, 1]),
 90764161042: tensor([1, 2, 3, 4, 5])}

In [88]:
x

tensor([1, 2, 3, 4, 5])

In [89]:
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [90]:
x_ptr.id_at_location

67898169666

In [91]:
x.id

67898169666

In [92]:
x.owner

<VirtualWorker id:me #objects:0>

In [93]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [94]:
x_pointer = x_ptr.get()
x_pointer

tensor([1, 2, 3, 4, 5])

In [95]:
bob._objects

{77944742512: tensor([1, 1, 1, 1, 1]), 90764161042: tensor([1, 2, 3, 4, 5])}

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# create a second Virtual worker and call it Alice
alice = sy.VirtualWorker(hook, id="alice")

In [105]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice
j = th.tensor([1,2,3,4,5])
b_ptr = j.send(bob,alice)
#a_ptr = j.send(alice)
print(b_ptr)
#print(a_ptr)


(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:59613166170 -> bob:32126058592]
	-> (Wrapper)>[PointerTensor | me:36897731565 -> alice:1579886185]


In [100]:
# notice what does the send fucntion on two workers return; a multi-pointer
b_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:23892043864 -> bob:43868389967]
	-> (Wrapper)>[PointerTensor | me:65394517259 -> alice:38998645555]

In [101]:
# what does .child on the pointer object return?
# what does .child.child on the pointer object return? 
b_ptr.child

MultiPointerTensor>{'bob': (Wrapper)>[PointerTensor | me:23892043864 -> bob:43868389967], 'alice': (Wrapper)>[PointerTensor | me:65394517259 -> alice:38998645555]}

In [102]:
b_ptr.child.child

{'alice': (Wrapper)>[PointerTensor | me:65394517259 -> alice:38998645555],
 'bob': (Wrapper)>[PointerTensor | me:23892043864 -> bob:43868389967]}

In [103]:
# try the .get() on the pointer
get_value = b_ptr.get()
get_value

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [0]:
# try the .get(sum_results=True) on your pointer
get_value = b_ptr.get(sum_results=True)


In [107]:
get_value

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [109]:
x # a pointer to a remote tensor located at bob

(Wrapper)>[PointerTensor | me:96024817961 -> bob:94300710466]

In [110]:
y # a pointer to another remote tensor located at bob

(Wrapper)>[PointerTensor | me:94120363477 -> bob:66936719750]

In [111]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

(Wrapper)>[PointerTensor | me:14041668723 -> bob:73404132537]

In [112]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [113]:
z = th.add(x,y) # we can also use Torch operation on these tensors to be executed remotely
z

(Wrapper)>[PointerTensor | me:66827905630 -> bob:48574448055]

In [114]:
z = z.get() # yup! same results!
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [116]:
z = (x + y).sum()
z = z.get()
z

tensor(20., requires_grad=True)

In [0]:
z.backward() 

In [0]:
x = x.get()

In [119]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
import torch
from torch import nn

In [168]:
# create some toy data for our model
input_data = th.tensor([[1.,1],[0.5,1],[1,0],[0,0]], requires_grad=True)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True)
print(input_data)
print(output_data)


tensor([[1.0000, 1.0000],
        [0.5000, 1.0000],
        [1.0000, 0.0000],
        [0.0000, 0.0000]], requires_grad=True)
tensor([[1.],
        [1.],
        [0.],
        [0.]], requires_grad=True)


In [169]:
# send the data to bob
input_pointer = input_data.send(bob)
output_pointer = output_data.send(bob)
input_pointer

(Wrapper)>[PointerTensor | me:29923383183 -> bob:59471591215]

In [170]:
output_pointer

(Wrapper)>[PointerTensor | me:61138658454 -> bob:46215763589]

In [172]:
# Weights and biases
w = torch.randn(1,2, requires_grad=True)
b = torch.randn(1, requires_grad=True)
w_ptr = w.send(bob)
print(w)
print(b)

tensor([[-1.8823,  0.5457]], requires_grad=True)
tensor([0.6780], requires_grad=True)


In [0]:
def model(x):
    return x @ w.t() + b
  


In [0]:
#mean squared error
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [175]:

# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction

#    2- calcualte the loss (a mean square loss)

#    3- backpropogate using the backword() function

#    4- DO NOT forget to clear your gradients after updating the weights

#    weights.data.sum_(weights.grad * lr)

for i in range(100):
    preds = model(input_data)
    loss = mse(preds, output_data)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()
 
# Calculate loss
preds = model(input_data)
loss = mse(preds, output_data)
print('Loss: ',loss)

# Print predictions
print('Prediction: ',preds)
# Print output
print('Output: ',output_data)




 



Loss:  tensor(1.2879, grad_fn=<DivBackward0>)
Prediction:  tensor([[-0.6544],
        [ 0.2860],
        [-1.2013],
        [ 0.6795]], grad_fn=<AddBackward0>)
Output:  tensor([[1.],
        [1.],
        [0.],
        [0.]], requires_grad=True)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{23831414651: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{92212512027: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [0]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects  # some error from Jupyter is casuing this to appear!

{49166137090: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [0]:
bob = bob.clear_objects() # erase force 
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects # notice that there is only a single tensor in bob

{17426510898: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [0]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.0860)
tensor(0.0586)
tensor(0.0402)
tensor(0.0278)
tensor(0.0194)
tensor(0.0136)
tensor(0.0096)
tensor(0.0069)
tensor(0.0049)
tensor(0.0036)
tensor(0.0026)
tensor(0.0019)
tensor(0.0014)
tensor(0.0010)
tensor(0.0008)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [0]:
train()

tensor(0.0824, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(5.0509e-06, requires_grad=True)
tensor(1.8660e-07, requires_grad=True)
tensor(1.2442e-07, requires_grad=True)
tensor(1.0333e-07, requires_grad=True)
tensor(8.5991e-08, requires_grad=True)
tensor(7.1573e-08, requires_grad=True)
tensor(5.9562e-08, requires_grad=True)
tensor(4.9569e-08, requires_grad=True)
tensor(4.1269e-08, requires_grad=True)
tensor(3.4340e-08, requires_grad=True)
tensor(2.8568e-08, requires_grad=True)
tensor(2.3789e-08, requires_grad=True)
tensor(1.9802e-08, requires_grad=True)
tensor(1.6471e-08, requires_grad=True)
tensor(1.3715e-08, requires_grad=True)
tensor(1.1405e-08, requires_grad=True)
tensor(9.4892e-09, requires_grad=True)
tensor(7.9073e-09, requires_grad=True)
tensor(6.5776e-09, requires_grad=True)
tensor(5.4857e-09, requires_grad=True)
tensor(4.5577e-09, requires_grad=True)
tensor(3.8027e-09, requires_grad=True)
tensor(3.1617e-09, requires_grad=True)
tensor(2.6215e-09, requires_grad=

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:bob #tensors:0>

<VirtualWorker id:alice #tensors:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [0]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [0]:
y = x + x

In [0]:
y

(Wrapper)>[PointerTensor | me:48924169518 -> alice:48924169518]

In [0]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5]),
 48924169518: tensor([ 2,  4,  6,  8, 10])}

In [0]:
alice._objects

{48924169518: (Wrapper)>[PointerTensor | alice:48924169518 -> bob:48924169518],
 55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{40541026396: (Wrapper)>[PointerTensor | alice:40541026396 -> bob:10539507281]}

In [0]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:40541026396 -> bob:10539507281]

In [0]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{28843833290: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{88814770977: (Wrapper)>[PointerTensor | alice:88814770977 -> bob:28843833290]}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{81966670653: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

In [0]:
x.move(alice)

(Wrapper)>[PointerTensor | me:547575813 -> alice:547575813]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{547575813: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{75487377866: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{94092707138: (Wrapper)>[PointerTensor | alice:94092707138 -> bob:75487377866]}

In [0]:
x.remote_get()

(Wrapper)>[PointerTensor | me:87205391815 -> alice:94092707138]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

In [0]:
x.move(bob)

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [0]:
x

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [0]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}